In [1]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder.master("local[*]") \
    .appName("Create Iceberg Table") \
    .config("spark.jars", "/home/ubuntu/Downloads/postgresql-42.6.0.jar") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.4.1") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", "./iceberg-partition-col") \
    .config("spark.hadoop.iceberg.writer.spec-id", "-1") \
    .getOrCreate()




23/11/14 16:02:55 WARN Utils: Your hostname, ubuntu-ThinkPad-T480 resolves to a loopback address: 127.0.1.1; using 192.168.1.111 instead (on interface wlp3s0)
23/11/14 16:02:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.4_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2eb8d8b7-ac2a-4b36-9911-65982d562cc6;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.4.1 in central
:: resolution report :: resolve 235ms :: artifacts dl 9ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.4.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.sp

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, BooleanType

# Define the schema for the data
schema = StructType([
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("date_of_birth", DateType(), True),
    StructField("gender", StringType(), True),
    StructField("address", StringType(), True),
    StructField("phone_number", StringType(), True),
    StructField("email", StringType(), True),
    StructField("diagnosis", StringType(), True),
    StructField("admission_date", DateType(), True),
    StructField("discharged", BooleanType(), True)
])
patient_df = spark.read.format("csv").option("delimeter", ",").option("header", True).schema(schema).load("data.csv")

In [ ]:
patient_df.repartition(1000).rdd.getNumPartitions()

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year

patient_df = patient_df.withColumn("month", month("admission_date"))
patient_df = patient_df.withColumn("year", year("admission_date"))
patient_df = patient_df.withColumn("day", dayofmonth("admission_date"))

In [ ]:

import random
from pyspark.sql.functions import udf

random_id_gen = udf(lambda: "PRAC"+str(random.randint(10000,99999)))
patient_df = patient_df.withColumn("practitioner", random_id_gen())
patient_df.repartition(1000).writeTo("local.patient_iceberg").createOrReplace()

In [ ]:
spark.sql("ALTER TABLE local.patient_iceberg ADD PARTITION FIELD year")
spark.sql("ALTER TABLE local.patient_iceberg ADD PARTITION FIELD month")
spark.sql("ALTER TABLE local.patient_iceberg ADD PARTITION FIELD day")
spark.sql("ALTER TABLE local.patient_iceberg ADD PARTITION FIELD diagnosis")
spark.sql("ALTER TABLE local.patient_iceberg ADD PARTITION FIELD gender")

In [ ]:
from pyspark.sql.types import FloatType
from faker import Faker

fake = Faker()

practitioner_schema = StructType([
    StructField("practitioner_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("specialty", StringType(), True),
    StructField("experience_years", FloatType(), True),
    StructField("location", StringType(), True),
    StructField("email", StringType(), True)
])

# Generate Fake practitioners
num_practitioners = 10000

practitioners = []
for id,_ in enumerate(range(num_practitioners)):
    practitioner_id = "PRAC" + str(num_practitioners + id)
    name = fake.name()
    specialty = fake.random.choice(["Cardiologist", "Dermatologist", "Pediatrician", "Orthopedic Surgeon", "Psychiatrist"])
    experience_years = float(fake.random.randrange(1,20))
    address = fake.address().replace('\n', '').replace(',', '')
    email = fake.email()

    practitioners.append((practitioner_id, name, specialty, experience_years, address, email))

# Create DataFrame from practitioners
practitioner_df = spark.createDataFrame(practitioners, schema=practitioner_schema)
practitioner_df.repartition(100).writeTo("local.practitioner_iceberg").createOrReplace()

In [3]:
import time 

s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg WHERE admission_date REGEXP '01-21'")
print(time.time() - s)

0.07547354698181152


In [4]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg WHERE admission_date REGEXP '2020-01-21' and diagnosis='Fever' and gender='Male' and first_name='Steven'")
print(time.time() - s)

0.12068796157836914


In [5]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg WHERE admission_date REGEXP '01-21' and diagnosis='Fever' and gender='Male'")
print(time.time() - s)

0.055736541748046875


In [6]:
s = time.time()
spark.sql("SELECT diagnosis, COUNT(diagnosis) as diagnosis_count FROM local.patient_iceberg GROUP BY diagnosis")
print(time.time() - s)

0.10874414443969727


In [7]:
s = time.time()
spark.sql("SELECT day, COUNT(day) as day_count FROM local.patient_iceberg GROUP BY day")
print(time.time() - s)

0.05083870887756348


In [8]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg INNER JOIN local.practitioner_iceberg on local.practitioner_iceberg.practitioner_id=local.patient_iceberg.practitioner")
print(time.time() - s)

0.08525776863098145


In [9]:
s = time.time()
spark.sql("SELECT * FROM local.practitioner_iceberg;")
print(time.time() - s)

0.01864337921142578


In [10]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg WHERE diagnosis = 'Headache';")
print(time.time() - s)

0.030848264694213867


In [11]:
s = time.time()
spark.sql("SELECT * FROM local.practitioner_iceberg WHERE experience_years > 10;")
print(time.time() - s)

0.030420303344726562


In [12]:
s = time.time()
spark.sql("SELECT gender, COUNT(*) FROM local.patient_iceberg GROUP BY gender;")
print(time.time() - s)

0.03101968765258789


In [13]:
s = time.time()
spark.sql("SELECT specialty, AVG(experience_years) FROM local.practitioner_iceberg GROUP BY specialty;")
print(time.time() - s)

0.03399229049682617


In [14]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg WHERE admission_date < '2023-01-01';")
print(time.time() - s)

0.03151512145996094


In [15]:
s = time.time()
spark.sql("SELECT * FROM local.practitioner_iceberg WHERE location LIKE '%City%';")
print(time.time() - s)

0.044147491455078125


In [16]:
s = time.time()
spark.sql("SELECT diagnosis, COUNT(*) FROM local.patient_iceberg GROUP BY diagnosis;")
print(time.time() - s)

0.0249483585357666


In [17]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg p JOIN local.practitioner_iceberg pr ON p.address = pr.location;")
print(time.time() - s)

0.04306387901306152


In [18]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg WHERE MONTH(admission_date) = 5;")
print(time.time() - s)

0.04182863235473633


In [19]:
s = time.time()
spark.sql("SELECT * FROM local.practitioner_iceberg WHERE experience_years BETWEEN 5 AND 15;")
print(time.time() - s)

0.020786285400390625


In [20]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg p JOIN local.practitioner_iceberg pr ON p.diagnosis = pr.specialty;")
print(time.time() - s)

0.03401327133178711


In [22]:
s = time.time()
spark.sql("SELECT YEAR(admission_date), COUNT(*) FROM local.patient_iceberg GROUP BY admission_date;")
print(time.time() - s)

0.05071377754211426


In [23]:
s = time.time()
spark.sql("SELECT * FROM local.practitioner_iceberg ORDER BY experience_years DESC LIMIT 1;")
print(time.time() - s)

0.04016900062561035


In [24]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg WHERE discharged = false;")
print(time.time() - s)

0.022139310836791992


In [25]:
s = time.time()
spark.sql("SELECT * FROM local.practitioner_iceberg WHERE email LIKE '%.com';")
print(time.time() - s)

0.016591310501098633


In [26]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg WHERE practitioner = '123';")
print(time.time() - s)

0.019825220108032227


In [27]:
s = time.time()
spark.sql("SELECT * FROM local.practitioner_iceberg WHERE specialty = 'Cardiologist' AND location = 'Hospital A';")
print(time.time() - s)

0.017183780670166016


In [28]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg p JOIN local.practitioner_iceberg pr ON p.gender = pr.name;")
print(time.time() - s)

0.026653528213500977


In [29]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg INNER JOIN local.practitioner_iceberg ON local.patient_iceberg.practitioner = local.practitioner_iceberg.practitioner_id;")
print(time.time() - s)

0.060747623443603516


In [30]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg RIGHT JOIN local.practitioner_iceberg ON local.patient_iceberg.practitioner = local.practitioner_iceberg.practitioner_id;")
print(time.time() - s)

0.02217698097229004


In [31]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg FULL OUTER JOIN local.practitioner_iceberg ON local.patient_iceberg.practitioner = local.practitioner_iceberg.practitioner_id;")
print(time.time() - s)

0.02578878402709961


In [32]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg CROSS JOIN local.practitioner_iceberg;")
print(time.time() - s)

0.020962953567504883


In [33]:
s = time.time()
spark.sql("SELECT * FROM local.patient_iceberg p1 INNER JOIN local.patient_iceberg p2 ON p1.practitioner = p2.practitioner AND p1.first_name != p2.first_name;")
print(time.time() - s)

0.050672292709350586
